In [4]:
import os 
import numpy as np
import tensorflow as tf
import cv2
import xml.etree.ElementTree as ET
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Carregar e Processar os Dados

In [5]:
def parse_xml(xml_file):
    tree = ET.parse(xml_file) #Carrega e analisa o arquivo XML.
    root = tree.getroot()
    boxes = []
    labels = []
    
    for obj in root.findall('object'):#Encontra todos os elementos <object> no XML, que representam as caixas delimitadoras.
        label = obj.find('name').text #Obtém o rótulo da caixa (livre, ocupada, parcialmente livre).
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text) #Obtém as coordenadas da caixa delimitadora.
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        boxes.append((xmin, ymin, xmax, ymax))
        labels.append(label)
    
    return boxes, labels


In [6]:
def load_data(image_folder, xml_folder):
    images = []
    labels = []
    #Para cada imagem, lê o arquivo XML correspondente para obter as caixas delimitadoras e os rótulos.
    for image_file in os.listdir(image_folder):
        if not image_file.endswith('.png'):
            continue
            
        image_path = os.path.join(image_folder, image_file)
        xml_path = os.path.join(xml_folder, image_file.replace('.jpg', '.xml'))
        
        image = cv2.imread(image_path)
        boxes, image_labels = parse_xml(xml_path)
        
        for box, label in zip(boxes, image_labels):
            xmin, ymin, xmax, ymax = box
            cropped_image = image[ymin:ymax, xmin:xmax]
            cropped_image = cv2.resize(cropped_image, (64, 64))
            images.append(cropped_image)
            labels.append(label)
    
    images = np.array(images) / 255.0
    labels = np.array(labels)
    
    return images, labels


# Modelo

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes de rótulos
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
